# Information Retrieval Project

Authors: Delia Mennitti - 19610, Letizia Meroi - 19041, Sara Napolitano - 24656 

# Introduction

Modern information retrieval systems face increasing challenges in serving multilingual and cross-lingual search needs. While traditional **sparse retrieval methods** like BM25 have been the industry standard for decades, they fundamentally rely on lexical overlap between queries and documents, limiting their effectiveness when dealing with semantic variations, paraphrases, or—most critically—different languages.

Recent advances in deep learning and multilingual language models have enabled **dense retrieval methods** that represent texts as continuous semantic vectors in a shared embedding space. These methods promise to capture semantic similarity across languages and overcome the vocabulary mismatch problem inherent to sparse approaches.

## Project Objectives

This project systematically investigates and compares the effectiveness of dense retrieval versus traditional sparse retrieval for multilingual information retrieval tasks. Specifically, we:

1. **Establish a baseline** using **BM25**, the most widely adopted sparse lexical retrieval method
2. **Implement dense retrieval** using **LaBSE** (Language-agnostic BERT Sentence Embedding), a state-of-the-art multilingual embedding model
3. **Develop a hybrid approach** that combines lexical and semantic signals through score fusion with language-specific optimization
4. **Evaluate comprehensively** across 5 diverse languages in both monolingual and cross-lingual settings using standardized metrics (Recall@10 and MRR@10)
5. **Analyze language-specific performance** to understand which retrieval paradigm works best for different language families and retrieval scenarios

## Research Questions

Our analysis addresses three key research questions:

- **RQ1**: How does dense semantic retrieval (LaBSE) compare to sparse lexical retrieval (BM25) for monolingual information retrieval?
- **RQ2**: What performance gains does dense retrieval provide for cross-lingual tasks where queries and documents are in different languages?
- **RQ3**: Can a hybrid approach combining both methods outperform either individual method, and what is the optimal balance between lexical and semantic signals for different languages?

## Technical Approach

The project implements a complete retrieval pipeline including:
- **Data processing** from the SWIM-IR multilingual dataset
- **Sparse retrieval** using BM25 with language-specific tokenization
- **Dense retrieval** using LaBSE embeddings and cosine similarity search
- **Hybrid fusion** with validation-based α tuning for optimal weighting
- **Comprehensive evaluation** framework for reproducible experiments

# Dataset and Task Description

For this project, we use the **SWIM-IR dataset**, which is described in detail in the paper *"Leveraging LLMs for Synthesizing Training Data Across Many Languages in Multilingual Dense Retrieval"* by Nandan Thakur, Jianmo Ni, Gustavo Hernández Ábrego, John Wieting, Jimmy Lin, and Daniel Cer.

We focus on a **cross-lingual Information Retrieval (IR) task** using the SWIM-IR dataset.

Given an English query, the objective is to **retrieve the relevant Wikipedia passage written in another language**. Each query has exactly one associated relevant passage, enabling **automatic and reproducible evaluation** of retrieval performance.

The dataset contains three main splits:
- **monolingual**: Query and document in the same language
- **cross_lingual**: Query in an other language, document in english (17 languages)
- **cross_lingual_ext**: Extended cross-lingual split focusing on Indic languages (16 languages)

This setup allows us to evaluate both monolingual and cross-lingual retrieval capabilities of different methods.

We selected a diverse set of 5 languages representing different language families:
- **German (de)**: Germanic language family
- **French (fr)**: Romance language family
- **Spanish (es)**: Romance language family
- **English (en)**: Germanic language family
- **Chinese (zh)**: Sino-Tibetan language family (only cross lingual)

This selection provides coverage across both European languages and non-alphabetic writing systems.

In [ ]:
import os
import json
import gzip
from collections import defaultdict
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast
from rank_bm25 import BM25Okapi
import jieba
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [ ]:
# Base data directory
BASE_DATA_DIR = "data/swim_ir_v1/swim_ir_v1"

# Exploratory Data Analysis

Before implementing our retrieval methods, we analyze the SWIM-IR dataset to understand its key characteristics: structure, scale, and text properties.

In [ ]:
# Dataset overview and corpus statistics
sns.set_style("whitegrid")

# Get structure and count corpus sizes
dataset_structure = {}
corpus_sizes = []

for split in os.listdir(BASE_DATA_DIR):
    split_path = os.path.join(BASE_DATA_DIR, split)
    if os.path.isdir(split_path) and not split.startswith('.'):
        languages = [d for d in os.listdir(split_path) 
                    if os.path.isdir(os.path.join(split_path, d)) and not d.startswith('.')]
        dataset_structure[split] = sorted(languages)
        
        # Count items per language
        for lang in languages:
            lang_path = os.path.join(split_path, lang, "train.jsonl")
            if os.path.isfile(lang_path):
                with open(lang_path, 'r', encoding='utf-8') as f:
                    count = sum(1 for _ in f)
                corpus_sizes.append({'split': split, 'language': lang, 'num_items': count})

df_sizes = pd.DataFrame(corpus_sizes)

print("="*70)
print("SWIM-IR DATASET OVERVIEW")
print("="*70)
print(f"\nSplits: {len(dataset_structure)} ({', '.join(dataset_structure.keys())})")
print(f"Total languages: {len(set([l for ls in dataset_structure.values() for l in ls]))}")
print(f"Total query-document pairs: {df_sizes['num_items'].sum():,}")
print(f"\nBy split:")
for split in dataset_structure:
    split_total = df_sizes[df_sizes['split']==split]['num_items'].sum()
    print(f"  {split}: {len(dataset_structure[split])} languages, {split_total:,} pairs")

# Compact visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('SWIM-IR Dataset Structure and Scale', fontsize=14, fontweight='bold')

# Split distribution
ax1 = axes[0]
split_totals = df_sizes.groupby('split')['num_items'].sum()
ax1.bar(range(len(split_totals)), split_totals.values, color=['#3498db', '#e74c3c', '#2ecc71'], alpha=0.7)
ax1.set_xticks(range(len(split_totals)))
ax1.set_xticklabels(split_totals.index, rotation=15, ha='right')
ax1.set_ylabel('Query-Document Pairs')
ax1.set_title('Corpus Size by Split')
for i, v in enumerate(split_totals.values):
    ax1.text(i, v, f'{v/1e6:.1f}M', ha='center', va='bottom', fontweight='bold')

# Top languages
ax2 = axes[1]
top_10 = df_sizes.nlargest(10, 'num_items')
ax2.barh(range(len(top_10)), top_10['num_items']/1000, color='steelblue', alpha=0.7)
ax2.set_yticks(range(len(top_10)))
ax2.set_yticklabels([f"{r['language']}" for _, r in top_10.iterrows()])
ax2.set_xlabel('Query-Document Pairs (thousands)')
ax2.set_title('Top 10 Languages by Corpus Size')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Text length analysis
SAMPLE_SIZE = 500
SELECTED_LANGUAGES_EDA = ['de', 'fr', 'es', 'en', 'zh']
CORPUS_SPLITS = [d for d in os.listdir(BASE_DATA_DIR) if os.path.isdir(os.path.join(BASE_DATA_DIR, d))]

text_stats = []
for split in CORPUS_SPLITS:
    split_path = os.path.join(BASE_DATA_DIR, split)
    if not os.path.exists(split_path):
        continue
        
    for lang in SELECTED_LANGUAGES_EDA:
        lang_path = os.path.join(split_path, lang, "train.jsonl")
        if not os.path.isfile(lang_path):
            continue
            
        doc_lengths, query_lengths, title_lengths = [], [], []
        with open(lang_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= SAMPLE_SIZE:
                    break
                try:
                    item = ast.literal_eval(line.strip())
                    doc_lengths.append(len(item.get('text', '').split()))
                    query_lengths.append(len(item.get('query', '').split()))
                    title_lengths.append(len(item.get('title', '').split()))
                except:
                    continue
        
        if doc_lengths:
            text_stats.append({
                'split': split, 'language': lang,
                'avg_doc': np.mean(doc_lengths),
                'avg_query': np.mean(query_lengths),
                'avg_title': np.mean(title_lengths)
            })

df_text = pd.DataFrame(text_stats)

if len(df_text) > 0:
    print("="*70)
    print("TEXT LENGTH CHARACTERISTICS")
    print("="*70)
    print(f"\nDocuments: ~{df_text['avg_doc'].mean():.0f} words")
    print(f"Queries: ~{df_text['avg_query'].mean():.0f} words")
    print(f"Titles: ~{df_text['avg_title'].mean():.0f} words")
    print(f"\nDocuments are ~{df_text['avg_doc'].mean() / df_text['avg_query'].mean():.1f}x longer than queries")
    
    # Single compact visualization
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    comparison = df_text.groupby('language')[['avg_doc', 'avg_query']].mean()
    comparison.plot(kind='bar', ax=ax, color=['steelblue', 'coral'], width=0.7)
    ax.set_ylabel('Average Length (words)')
    ax.set_xlabel('Language')
    ax.set_title('Document vs Query Length by Language', fontweight='bold')
    ax.legend(['Document', 'Query'])
    ax.grid(axis='y', alpha=0.3)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Methodology

### Evaluation Metrics
We evaluate all retrieval methods using two standard information retrieval metrics computed at rank $K=10$:
- **Recall@10**: The fraction of queries for which the relevant document appears in the top-10 ranked results. This metric measures the system's ability to retrieve relevant documents without requiring them to be ranked first.
  
  $$\text{Recall@10} = \frac{\text{nr queries with relevant doc in top-10}}{\text{Total nr queries}}$$

- **MRR@10 (Mean Reciprocal Rank)**: The average reciprocal rank of the first relevant document for all queries, capped at rank 10. This metric rewards methods that rank relevant documents higher in the result list.
  
  $$\text{MRR@10} = \frac{1}{|Q|} \sum_{q=1}^{|Q|} \frac{1}{\text{rank}_q}$$
  
  where $\text{rank}_q$ is the position of the relevant document for query $q$ (or 0 if not in top-10).

Both metrics range from 0 to 1, with higher values indicating better performance.

### Retrieval Methods Compared
We compare three retrieval approaches:
1. **BM25 (Sparse Lexical Retrieval)**: A baseline probabilistic ranking function based on term frequency and inverse document frequency weighting.
2. **LaBSE (Dense Semantic Retrieval)**: A multilingual BERT-based embedding model that captures semantic similarity across languages without explicit fine-tuning.
3. **Hybrid (Score-based Fusion)**: A weighted combination of BM25 and LaBSE scores, with the weight parameter $\alpha$ tuned on a validation set to maximize performance.

For each method, we retrieve the top 10 documents per query and evaluate against the ground-truth relevant document.

# 1. Baseline: BM25 (Sparse Lexical Retrieval)

We implement **BM25 (Best Matching 25)**, a probabilistic ranking function based on the bag-of-words representation. BM25 is one of the most widely used sparse retrieval methods in information retrieval.

**Key characteristics:**
- Uses term frequency (TF) and inverse document frequency (IDF) weighting
- Language-specific tokenization (including jieba for Chinese)
- No semantic understanding - relies purely on lexical overlap
- Fast and efficient for large-scale retrieval

BM25 serves as our baseline to evaluate the effectiveness of dense retrieval methods.

In [ ]:
def encode_texts(texts, batch_size=1):
    """Encode texts to embeddings.
    EXTREMELY CONSERVATIVE: batch_size=1 (process one at a time)
    """
    if not globals().get('RUN_DENSE', False):
        raise RuntimeError('Dense encoding was disabled (RUN_DENSE=False)')
    
    if not texts:
        raise ValueError('No texts provided for encoding')
    
    try:
        print(f"  Encoding {len(texts)} texts with batch_size={batch_size}...")
        embeddings = model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True  # important for cosine similarity
        )
        # Force garbage collection after encoding
        gc.collect()
        return embeddings
    except MemoryError as me:
        print(f'[ERROR] MemoryError during encoding: {me}')
        print('[TIP] Memory exhausted - try disabling RUN_DENSE or reducing MAX_ITEMS to 2')
        gc.collect()
        raise
    except Exception as e:
        print(f'[ERROR] Error during encoding: {e}')
        gc.collect()
        raise

In [ ]:
# Global cache for scores
SCORE_CACHE = {}  # key: (split, lang) -> dict with bm25_scores, dense_scores, doc_ids, query_ids, qrels

# Run BM25 for each language folder under each split (cross_lingual, monolingual, etc.)
CORPUS_SPLITS = [d for d in os.listdir(BASE_DATA_DIR) if os.path.isdir(os.path.join(BASE_DATA_DIR, d))]

# Fixed set of languages to process (Italian not available in dataset)
SELECTED_LANGUAGES = ['de', 'fr', 'es', 'en', 'zh']  # German, French, Spanish, English, Chinese

# Start with 5 items - sklearn is more stable than FAISS
MAX_ITEMS = 5000 
K = 10  # top-K retrieval
CANDIDATE_N = 500  # candidates from BM25 and Dense (>= K)

# Robust JSONL loader
def load_jsonl_robust(path, max_items=MAX_ITEMS):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if max_items and i >= max_items:
                break
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                obj = ast.literal_eval(line)
            data.append(obj)
    return data

# Tokenizer
def tokenize(text, lang_code):
    if lang_code == "zh":
        return list(jieba.cut(text))
    else:
        return text.lower().split()

# Evaluation
def evaluate(retrieved, qrels, K=10):
    recalls = []
    rr_list = []
    for qid, top_docs in retrieved.items():
        relevant_doc = qrels[qid]
        recalls.append(1.0 if relevant_doc in top_docs[:K] else 0.0)
        try:
            rank = top_docs.index(relevant_doc) + 1
            rr_list.append(1.0 / rank)
        except ValueError:
            rr_list.append(0.0)
    return np.mean(recalls), np.mean(rr_list)

# Store results
results = []

print(f"Processing selected languages: {', '.join(SELECTED_LANGUAGES)}\n")

In [ ]:
BM25_CACHE_DIR = "artifacts_cache_bm25"
os.makedirs(BM25_CACHE_DIR, exist_ok=True)

def bm25_cache_path(split, lang, max_items, k, topn):
    return os.path.join(BM25_CACHE_DIR, f"{split}__{lang}__max{max_items}__k{k}__topn{topn}.json")

def load_bm25_from_disk(split, lang, max_items=MAX_ITEMS, k=K, topn=None):
    if topn is None:
        topn = max(CANDIDATE_N, K)
    path = bm25_cache_path(split, lang, max_items, k, topn)
    if not os.path.exists(path):
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_bm25_to_disk(split, lang, payload, max_items=MAX_ITEMS, k=K, topn=None):
    if topn is None:
        topn = payload.get("TOPN", max(CANDIDATE_N, K))
    path = bm25_cache_path(split, lang, max_items, k, topn)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f)



In [ ]:
# put true only if you really need the QxD BM25 matrix
STORE_BM25_MATRIX = False  


In [ ]:
for split in CORPUS_SPLITS:
    split_path = os.path.join(BASE_DATA_DIR, split)
    
    for lang_code in os.listdir(split_path):
        # Only process selected languages
        if lang_code not in SELECTED_LANGUAGES:
            continue
            
        lang_dir = os.path.join(split_path, lang_code)
        lang_path = os.path.join(lang_dir, "train.jsonl")
        if not os.path.isfile(lang_path):
            continue
            
        print(f"\n{'='*60}")
        print(f"Processing {split}/{lang_code}")
        print('='*60)

        TOPN = max(CANDIDATE_N, K)
        cached = load_bm25_from_disk(split, lang_code, MAX_ITEMS, K, TOPN)

        if cached is not None:
            print("BM25 cache hit – loading top-K from disk (skip BM25 computation)")

            # put in RAM cache for later hybrid/rerank
            SCORE_CACHE[(split, lang_code)] = SCORE_CACHE.get((split, lang_code), {})
            SCORE_CACHE[(split, lang_code)].update(cached)

            # Recompute metrics quickly from cached top-K
            recall, mrr = evaluate(cached["bm25_topk_docs"], cached["qrels"], K=K)
            results.append({
                "split": split,
                "language": lang_code,
                "Recall@10": recall,
                "MRR@10": mrr,
                "method": "BM25"
            })
            continue

        data = load_jsonl_robust(lang_path, max_items=MAX_ITEMS)

        # Build documents, queries, qrels
        documents = {}
        queries = {}
        qrels = {}
        for item in data:
            doc_id = f"{lang_code}_{item['_id']}"
            lang_field = item.get("code", lang_code)
            documents[doc_id] = {"text": item.get("title", "") + " " + item.get("text", ""), "lang": lang_field}
            queries[doc_id] = item.get("query", "")
            qrels[doc_id] = doc_id

        # Tokenize corpus
        doc_ids = list(documents.keys())
        query_ids = list(queries.keys())

        tokenized_corpus = [tokenize(documents[doc_id]["text"], documents[doc_id]["lang"]) for doc_id in doc_ids]

        if len(tokenized_corpus) == 0:
            print("No documents found, skipping.")
            continue

        bm25 = BM25Okapi(tokenized_corpus)

        # Tokenize queries
        tokenized_queries = {qid: tokenize(q, documents[qid]["lang"]) for qid, q in queries.items()}

        # Optional: Build BM25 score matrix QxD
        Q = len(query_ids)
        D = len(doc_ids)
        bm25_scores_matrix = None
        if STORE_BM25_MATRIX:
            bm25_scores_matrix = np.zeros((Q, D), dtype=np.float32)

        # Retrieve top-K
        bm25_topk_docs = {}
        bm25_topk_scores = {}
        retrieved = {}
        TOPN = max(CANDIDATE_N, K)
        for i, qid in enumerate(query_ids):
            scores = np.asarray(bm25.get_scores(tokenized_queries[qid]), dtype=np.float32)
            if STORE_BM25_MATRIX:
                bm25_scores_matrix[i, :] = scores

            top_indices = scores.argsort()[-TOPN:][::-1]
            docs = [doc_ids[j] for j in top_indices]
            scs  = [float(scores[j]) for j in top_indices]

            retrieved[qid] = docs
            bm25_topk_docs[qid] = docs
            bm25_topk_scores[qid] = scs

        # Evaluate
        recall, mrr = evaluate(retrieved, qrels, K=K)
        results.append({"split": split, "language": lang_code, "Recall@10": recall, "MRR@10": mrr, "method": "BM25"})
        print(f"[OK] BM25 - Recall@{K}: {recall:.4f}, MRR@{K}: {mrr:.4f}")
        
        # Save to cache
        key = (split, lang_code)
        SCORE_CACHE[key] = SCORE_CACHE.get(key, {})
        update_pack = {
            "bm25_topk_docs": bm25_topk_docs,
            "bm25_topk_scores": bm25_topk_scores,
            "doc_ids": doc_ids,
            "query_ids": query_ids,
            "qrels": qrels
        }
        if STORE_BM25_MATRIX:
            update_pack["bm25"] = bm25_scores_matrix

        SCORE_CACHE[key].update(update_pack)


        # Save only BM25 top-K to disk, matrix is large and remains in RAM
        payload = {
            "doc_ids": doc_ids,
            "query_ids": query_ids,
            "qrels": qrels,
            "bm25_topk_docs": bm25_topk_docs,
            "bm25_topk_scores": bm25_topk_scores,
            "TOPN": int(TOPN)
        }
        save_bm25_to_disk(split, lang_code, payload, MAX_ITEMS, K, TOPN)
        print("BM25 top-K cached to disk.")
        gc.collect()

# Display summary table
print("\n" + "="*60)
print("SUMMARY RESULTS")
print("="*60)
df_results = pd.DataFrame(results)
print(f"\nTotal experiments: {len(df_results)}")


df_results

# 2. Dense Retrieval Using Multilingual Embeddings

## Approach

We implement a **dense retrieval system** using deep learning-based text embeddings. Unlike sparse methods like BM25, dense retrieval represents texts as continuous vectors in a high-dimensional space, enabling semantic similarity matching.

## Model Selection: LaBSE

We use **LaBSE** (Language-agnostic BERT Sentence Embedding) from the sentence-transformers library, which offers:

- **Multilingual support**: Pre-trained on 109 languages
- **Cross-lingual alignment**: Semantically similar texts in different languages map to nearby vectors
- **768-dimensional embeddings**: Rich semantic representations
- **Zero-shot capability**: No fine-tuning required for new language pairs

## Implementation Details

For similarity computation, we use **sklearn's cosine_similarity** instead of FAISS. This approach proved more stable and memory-efficient for our dataset size (1000 items per language), avoiding kernel crashes experienced with FAISS on M3 Mac hardware (16GB RAM).

The retrieval process:
1. Encode all documents into 768-dim vectors
2. Encode all queries into 768-dim vectors  
3. Compute cosine similarity matrix (queries × documents)
4. Rank documents by similarity scores
5. Retrieve top-K results for evaluation

## Alternative Models Considered

Several multilingual embedding models were systematically evaluated for suitability in cross-lingual information retrieval. LaBSE emerged as the preferred choice due to its strong cross-lingual alignment and consistent zero-shot performance across languages, eliminating the need for task-specific fine-tuning. While mSBERT offered advantages in terms of speed and reduced model size, its cross-lingual representations were comparatively weaker, making it less reliable for retrieval accuracy. A fine-tuned XLM-R model was also considered because of its flexibility; however, it requires additional pooling strategies and careful tuning to perform effectively for information retrieval tasks. Overall, LaBSE provided the best balance of performance and operational simplicity for the project’s multilingual requirements.


In [ ]:
# Dense retrieval: Using sklearn instead of FAISS (more stable)
RUN_DENSE = True  # set to False to skip dense retrieval

if RUN_DENSE:
    try:
        from sentence_transformers import SentenceTransformer
        from sklearn.metrics.pairwise import cosine_similarity
        import numpy as np
        model = SentenceTransformer("sentence-transformers/LaBSE")
        print('[OK] LaBSE model loaded')
        print('[OK] Using sklearn cosine_similarity (more stable than FAISS)')
    except Exception as e:
        print('Failed to load dense model:', e)
        RUN_DENSE = False
else:
    print('Dense retrieval disabled (RUN_DENSE=False)')

In [ ]:
import os, json, tempfile

DENSE_CACHE_DIR = "artifacts_cache_dense"
os.makedirs(DENSE_CACHE_DIR, exist_ok=True)

def dense_cache_path(split, lang):
    return os.path.join(DENSE_CACHE_DIR, f"{split}__{lang}__max{MAX_ITEMS}__k{K}.json")

def load_dense_from_disk(split, lang):
    path = dense_cache_path(split, lang)
    if not os.path.exists(path):
        return None
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except json.JSONDecodeError:
        # Corrupted file - rename to .bad and treat as cache miss
        bad = path + ".bad"
        try:
            os.replace(path, bad)
        except Exception:
            pass
        print(f"[WARN] Dense cache corrupt -> renamed to {bad}. Recompute...")
        return None

def save_dense_to_disk(split, lang, payload):
    os.makedirs(DENSE_CACHE_DIR, exist_ok=True)
    path = dense_cache_path(split, lang)

    # Atomic write, avoid partial files
    fd, tmp_path = tempfile.mkstemp(dir=DENSE_CACHE_DIR, suffix=".tmp")
    try:
        with os.fdopen(fd, "w", encoding="utf-8") as f:
            json.dump(payload, f)
            f.flush()
            os.fsync(f.fileno())
        os.replace(tmp_path, path)
    finally:
        if os.path.exists(tmp_path):
            try:
                os.remove(tmp_path)
            except Exception:
                pass


In [ ]:
for split in CORPUS_SPLITS:
    split_path = os.path.join(BASE_DATA_DIR, split)
    
    for lang_code in os.listdir(split_path):
        # Only process selected languages
        if lang_code not in SELECTED_LANGUAGES:
            continue

        print(f"\n{'='*60}")
        print(f"Processing {split}/{lang_code}")
        print('='*60)

        # Check cache
        cached_dense = load_dense_from_disk(split, lang_code)
        if cached_dense is not None:
            print("Dense cache hit – loading precomputed top-N...")

            key = (split, lang_code)
            SCORE_CACHE[key] = SCORE_CACHE.get(key, {})
            SCORE_CACHE[key].update(cached_dense)
            
            # Evaluate cached results
            recall, mrr = evaluate(cached_dense["dense_topk_docs"], cached_dense["qrels"], K=K)
            results.append({
                "split": split,
                "language": lang_code,
                "Recall@10": recall,
                "MRR@10": mrr,
                "method": "LaBSE"
            })
            print(f"[OK] LaBSE - Recall@{K}: {recall:.4f}, MRR@{K}: {mrr:.4f}")
            continue

        print("Cache miss – computing scores...")

        lang_dir = os.path.join(split_path, lang_code)
        lang_path = os.path.join(lang_dir, "train.jsonl")
        if not os.path.isfile(lang_path):
            continue
            
        
        data = load_jsonl_robust(lang_path, max_items=MAX_ITEMS)

        # Build documents, queries, qrels
        documents = {}
        queries = {}
        qrels = {}
        for item in data:
            doc_id = f"{lang_code}_{item['_id']}"
            lang_field = item.get("code", lang_code)
            documents[doc_id] = {"text": item.get("title", "") + " " + item.get("text", ""), "lang": lang_field}
            queries[doc_id] = item.get("query", "")
            qrels[doc_id] = doc_id

        
        doc_ids = list(documents.keys())
        query_ids = list(queries.keys())


        # Dense retrieval for this language using sklearn (no FAISS!)
        if globals().get('RUN_DENSE', False):
            try:
                # Encode documents
                doc_texts = [documents[doc_id]["text"] for doc_id in doc_ids]
                print(f"\n[DENSE] Encoding {len(doc_texts)} documents...")
                doc_embeddings = encode_texts(doc_texts)
                print(f"[OK] Document embeddings shape: {doc_embeddings.shape}")

                # Encode queries
                query_id = list(queries.keys())
                query_texts = [queries[qid] for qid in query_ids]
                print(f"\n[DENSE] Encoding {len(query_texts)} queries...")
                query_embeddings = encode_texts(query_texts)
                print(f"[OK] Query embeddings shape: {query_embeddings.shape}")

                # Use sklearn cosine_similarity instead of FAISS
                print(f"\n[DENSE] Computing cosine similarities...")
                # Compute similarity matrix: queries x documents
                similarities = cosine_similarity(query_embeddings, doc_embeddings)
                print(f"[OK] Similarity matrix shape: {similarities.shape}")
                
                # Get top-K for each query
                dense_topk_docs = {}
                dense_topk_scores = {}
                TOPN = max(CANDIDATE_N, K)
                for i, qid in enumerate(query_ids):
                    # Get similarities for this query
                    query_sims = similarities[i]
                    # Get top-K indices
                    top_indices = query_sims.argsort()[-TOPN:][::-1]

                    docs = [doc_ids[idx] for idx in top_indices]
                    scores = [float(query_sims[idx]) for idx in top_indices]

                    dense_topk_docs[qid] = docs
                    dense_topk_scores[qid] = scores
                    print(f"  Query {i+1}/{len(query_ids)}: done")

                # Evaluate
                recall, mrr = evaluate(dense_topk_docs, qrels, K=K)
                results.append({
                    "split": split,
                    "language": lang_code,
                    "Recall@10": recall,
                    "MRR@10": mrr,
                    "method": "LaBSE"
                })
                print(f"\n[OK] LaBSE - Recall@{K}: {recall:.4f}, MRR@{K}: {mrr:.4f}")
                
                # Save dense matrix to cache
                dense_payload = {
                    "doc_ids": doc_ids,
                    "query_ids": query_ids,
                    "qrels": qrels,
                    "dense_topk_docs": dense_topk_docs,
                    "dense_topk_scores": dense_topk_scores,
                    "TOPN": int(TOPN)
                }

                key = (split, lang_code)
                SCORE_CACHE[key] = SCORE_CACHE.get(key, {})
                SCORE_CACHE[key].update(dense_payload)

                save_dense_to_disk(split, lang_code, dense_payload)
                print("Dense top-N cached to disk.")



                # Clean up
                del doc_embeddings, query_embeddings, similarities
                gc.collect()
                
            except Exception as e:
                print(f"\n[ERROR] Dense retrieval failed for {split}/{lang_code}: {e}")
                print("Continuing with next language...")
                import traceback
                traceback.print_exc()
                gc.collect()
        
        gc.collect()

# Display summary table
print("\n" + "="*60)
print("SUMMARY RESULTS")
print("="*60)
df_results = pd.DataFrame(results)
print(f"\nTotal experiments: {len(df_results)}")

df_results

In [ ]:
import copy
import time

# copy and freeze results and use them later
results_frozen = copy.deepcopy(results)
df_results_frozen = df_results.copy(deep=True)

# save frozen results with timestamp
ts = time.strftime("%Y%m%d_%H%M%S")
df_results_frozen.to_pickle(f"df_results_frozen_{ts}.pkl")

print("Base results frozen and saved")

# Comparison between BM25 and LaBSE

In [ ]:
# Analyze and compare BM25 vs LaBSE (Dense BERT-based retrieval)
import matplotlib.pyplot as plt
import seaborn as sns

if 'df_results_frozen' in globals() and not df_results_frozen.empty:
    print("="*70)
    print("DETAILED RESULTS COMPARISON: BM25 vs LaBSE (Dense BERT)")
    print("="*70)
    
    # Show all results
    print("\n[RESULTS] All Results:")
    print(df_results_frozen.to_string(index=False))
    
    # Compare by method
    print("\n" + "="*70)
    print("AVERAGE PERFORMANCE BY METHOD")
    print("="*70)
    method_avg = df_results_frozen.groupby('method')[['Recall@10', 'MRR@10']].mean()
    print(method_avg)
    
    # Compare by language
    print("\n" + "="*70)
    print("PERFORMANCE BY LANGUAGE")
    print("="*70)
    
    # Pivot table for easier comparison
    pivot_recall = df_results_frozen.pivot_table(
        values='Recall@10', 
        index=['split', 'language'], 
        columns='method'
    ).fillna(0)
    
    pivot_mrr = df_results_frozen.pivot_table(
        values='MRR@10', 
        index=['split', 'language'], 
        columns='method'
    ).fillna(0)
    
    print("\n[METRICS] Recall@10 by Language:")
    print(pivot_recall)
    
    print("\n[METRICS] MRR@10 by Language:")
    print(pivot_mrr)
    
    # Count wins
    if 'LaBSE' in df_results_frozen['method'].values and 'BM25' in df_results_frozen['method'].values:
        print("\n" + "="*70)
        print("HEAD-TO-HEAD COMPARISON")
        print("="*70)
        
        # Compare on Recall@10
        better_recall = (pivot_recall['LaBSE'] > pivot_recall['BM25']).sum()
        worse_recall = (pivot_recall['LaBSE'] < pivot_recall['BM25']).sum()
        tie_recall = (pivot_recall['LaBSE'] == pivot_recall['BM25']).sum()
        
        print(f"\n[COMPARISON] Recall@10 comparison:")
        print(f"  LaBSE wins: {better_recall}")
        print(f"  BM25 wins: {worse_recall}")
        print(f"  Ties: {tie_recall}")
        
        # Compare on MRR@10
        better_mrr = (pivot_mrr['LaBSE'] > pivot_mrr['BM25']).sum()
        worse_mrr = (pivot_mrr['LaBSE'] < pivot_mrr['BM25']).sum()
        tie_mrr = (pivot_mrr['LaBSE'] == pivot_mrr['BM25']).sum()
        
        print(f"\n[COMPARISON] MRR@10 comparison:")
        print(f"  LaBSE wins: {better_mrr}")
        print(f"  BM25 wins: {worse_mrr}")
        print(f"  Ties: {tie_mrr}")
        
        print("\n[INFO] LaBSE uses BERT-based dense embeddings (768-dim vectors)")
        print("[INFO] sklearn cosine_similarity for retrieval (no FAISS)")
        
        # Visualization plots
        print("\n" + "="*70)
        print("VISUALIZATIONS")
        print("="*70)
        
        # Set style
        sns.set_style("whitegrid")
        
        # Create figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('BM25 vs LaBSE Performance Comparison', fontsize=16, fontweight='bold')
        
        # 1. Bar chart: Average metrics by method
        ax1 = axes[0, 0]
        method_avg.plot(kind='bar', ax=ax1, color=['#2ecc71', '#e74c3c'])
        ax1.set_title('Average Performance by Method', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Method')
        ax1.set_ylabel('Score')
        ax1.legend(title='Metrics')
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
        ax1.grid(axis='y', alpha=0.3)
        
        # 2. Grouped bar chart: MRR@10 by language
        ax2 = axes[0, 1]
        pivot_mrr.plot(kind='bar', ax=ax2, width=0.8)
        ax2.set_title('MRR@10 by Language', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Language')
        ax2.set_ylabel('MRR@10')
        ax2.legend(title='Method')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
        ax2.grid(axis='y', alpha=0.3)
        
        # 3. Scatter plot: BM25 vs LaBSE (Recall)
        ax3 = axes[1, 0]
        for idx, row in pivot_mrr.iterrows():
            split, lang = idx if isinstance(idx, tuple) else (idx, idx)
            ax3.scatter(row['BM25'], row['LaBSE'], s=100, alpha=0.6)
            ax3.annotate(lang, (row['BM25'], row['LaBSE']), 
                        fontsize=8, ha='right', va='bottom')
        
        # Add diagonal line
        max_val = max(pivot_mrr.max().max(), 1.0)
        ax3.plot([0, max_val], [0, max_val], 'k--', alpha=0.3, label='Equal performance')
        ax3.set_xlim(0, max_val * 1.05)
        ax3.set_ylim(0, max_val * 1.05)
        ax3.set_xlabel('BM25 MRR@10')
        ax3.set_ylabel('LaBSE MRR@10')
        ax3.set_title('BM25 vs LaBSE Performance (MRR@10)', fontsize=12, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 4. Win/Loss pie chart
        ax4 = axes[1, 1]
        win_data = [better_mrr, worse_mrr, tie_mrr]
        labels = [f'LaBSE wins\n({better_mrr})', f'BM25 wins\n({worse_mrr})', f'Ties\n({tie_mrr})']
        colors = ['#2ecc71', '#e74c3c', '#95a5a6']
        wedges, texts, autotexts = ax4.pie(win_data, labels=labels, colors=colors, 
                                             autopct='%1.1f%%', startangle=90)
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        ax4.set_title('Head-to-Head Wins (MRR@10)', fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Additional plot: Language-wise comparison
        if len(pivot_recall) > 1:
            fig2, ax = plt.subplots(figsize=(12, 6))
            x = range(len(pivot_recall))
            width = 0.35
            
            ax.bar([i - width/2 for i in x], pivot_recall['BM25'], width, label='BM25', alpha=0.8)
            ax.bar([i + width/2 for i in x], pivot_recall['LaBSE'], width, label='LaBSE', alpha=0.8)
            
            ax.set_xlabel('Language (Split)', fontsize=12)
            ax.set_ylabel('Recall', fontsize=12)
            ax.set_title('Side-by-Side Comparison: BM25 vs LaBSE', fontsize=14, fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels([f"{idx[1]}\n({idx[0]})" for idx in pivot_recall.index], 
                              rotation=45, ha='right')
            ax.legend()
            ax.grid(axis='y', alpha=0.3)
            
            plt.tight_layout()
            plt.show()
else:
    print("[WARNING] No results available. Run the main loop (cell 11) first.")

## 2. Hybrid Approach (BM25 + Dense Retrieval)

Another valuable direction is to combine the strengths of both sparse and dense methods.

**Approach:**
- Compute both BM25 scores and LaBSE similarity scores
- Combine scores using weighted linear combination: `score = α × BM25 + (1-α) × LaBSE`
- Tune α parameter on validation set
- Alternative: Use rank fusion methods (Reciprocal Rank Fusion)

**Rationale:**
- BM25 captures exact lexical matches (good for entity names, technical terms)
- Dense retrieval captures semantic similarity (good for paraphrases, cross-lingual)
- Hybrid methods often outperform either method alone

This approach has shown consistent improvements in recent IR research and TREC competitions.

In [ ]:
# safe copy for hybrid experiments
results_hybrid = copy.deepcopy(results_frozen)

In [ ]:
CANDIDATE_N = 500          # candidati da BM25 e Dense (>= K)
alpha_grid = np.linspace(0.0, 1.0, 11)
seed = 42
val_ratio = 0.2

def to_score_dict(top_docs, top_scores):
    """Convert two lists (docs, scores) to {doc: score}."""
    return {d: float(s) for d, s in zip(top_docs, top_scores)}

def minmax_scores(score_map):
    """
    Min-max normalize a dict {doc: score} to [0,1] over its values.
    Returns dict {doc: norm_score}.
    """
    if not score_map:
        return {}
    vals = np.array(list(score_map.values()), dtype=np.float32)
    vmin, vmax = float(vals.min()), float(vals.max())
    if vmax == vmin:
        return {d: 0.0 for d in score_map}
    return {d: (float(s) - vmin) / (vmax - vmin) for d, s in score_map.items()}

def hybrid_topk_for_query(qid, alpha, pack, K=10, candN=100):
    """
    Candidate-based hybrid for a single query id.
    Combines BM25 and Dense scores on the union of their top candN docs.
    """
    bm_docs = pack["bm25_topk_docs"].get(qid, [])[:candN]
    bm_scs  = pack["bm25_topk_scores"].get(qid, [])[:candN]
    de_docs = pack["dense_topk_docs"].get(qid, [])[:candN]
    de_scs  = pack["dense_topk_scores"].get(qid, [])[:candN]

    bm_map = to_score_dict(bm_docs, bm_scs)
    de_map = to_score_dict(de_docs, de_scs)

    cands = set(bm_map) | set(de_map)
    if not cands:
        return []

    # Normalize within candidate set (missing docs get score 0.0)
    bm_norm = minmax_scores({d: bm_map.get(d, 0.0) for d in cands})
    de_norm = minmax_scores({d: de_map.get(d, 0.0) for d in cands})

    # Same formula as full hybrid, but only on candidates
    hybrid = [(d, alpha * bm_norm[d] + (1.0 - alpha) * de_norm[d]) for d in cands]
    hybrid.sort(key=lambda x: x[1], reverse=True)

    return [d for d, _ in hybrid[:K]]

# Assumes you already have evaluate(retrieved, qrels, K)
# (the one you posted is fine)

# reset to avoid duplicates when re-running the cell
results_hybrid = copy.deepcopy(results_frozen) if "results_frozen" in globals() else []

# drop any previous hybrid rows if present
results_hybrid = [r for r in results_hybrid if not str(r.get("method","")).startswith("Hybrid")]


alpha_curve_results = []

for split in CORPUS_SPLITS:
    for lang_code in SELECTED_LANGUAGES:
        key = (split, lang_code)
        if key not in SCORE_CACHE:
            continue

        pack = SCORE_CACHE[key]
        needed = ["qrels", "bm25_topk_docs", "bm25_topk_scores", "dense_topk_docs", "dense_topk_scores"]
        if any(k not in pack for k in needed):
            # Not all components present for hybrid
            continue

        qrels = pack["qrels"]
        query_ids = list(qrels.keys())  # in your dataset qid==docid, so this is fine

        print(f"\n{'='*60}")
        print(f"Hybrid tuning for {split}/{lang_code}")
        print('='*60)

        # val/test split over query_ids
        rng = np.random.default_rng(seed)
        shuffled = query_ids.copy()
        rng.shuffle(shuffled)
        n_val = max(1, int(len(shuffled) * val_ratio))
        val_ids = shuffled[:n_val]
        test_ids = shuffled[n_val:] if n_val < len(shuffled) else shuffled[:]

        best_alpha = None
        best_val_mrr = -1.0
        best_val_recall = -1.0

        # ---- Tune alpha on VAL ----
        for alpha in alpha_grid:
            retrieved_val = {
                qid: hybrid_topk_for_query(qid, float(alpha), pack, K=K, candN=CANDIDATE_N)
                for qid in val_ids
            }

            recall_val, mrr_val = evaluate(retrieved_val, qrels, K=K)

            alpha_curve_results.append({
                "split": split,
                "language": lang_code,
                "alpha": float(alpha),
                "val_Recall@10": float(recall_val),
                "val_MRR@10": float(mrr_val),
                "val_ratio": float(val_ratio),
                "seed": int(seed),
                "candN": int(CANDIDATE_N),
            })

            # tie-break: higher MRR, then higher recall, then smaller alpha
            if (mrr_val > best_val_mrr) or \
               (mrr_val == best_val_mrr and recall_val > best_val_recall) or \
               (mrr_val == best_val_mrr and recall_val == best_val_recall and (best_alpha is None or alpha < best_alpha)):
                best_val_mrr = mrr_val
                best_val_recall = recall_val
                best_alpha = float(alpha)

        print(f"[TUNE] Best alpha={best_alpha:.2f} on VAL | Recall@{K}={best_val_recall:.4f}, MRR@{K}={best_val_mrr:.4f}")

        # ---- Evaluate on TEST with best alpha ----
        retrieved_test = {
            qid: hybrid_topk_for_query(qid, best_alpha, pack, K=K, candN=CANDIDATE_N)
            for qid in test_ids
        }

        recall_test, mrr_test = evaluate(retrieved_test, qrels, K=K)

        results_hybrid.append({
            "split": split,
            "language": lang_code,
            "Recall@10": float(recall_test),
            "MRR@10": float(mrr_test),
            "method": "Hybrid(tuned)",
            "alpha": float(best_alpha),
            "val_Recall@10": float(best_val_recall),
            "val_MRR@10": float(best_val_mrr),
            "candN": int(CANDIDATE_N),
        })

        print(f"[OK] Hybrid TEST - alpha={best_alpha:.2f} - Recall@{K}: {recall_test:.4f}, MRR@{K}: {mrr_test:.4f}")
        gc.collect()

df_results_hybrid = pd.DataFrame(results_hybrid)
df_alpha_curve = pd.DataFrame(alpha_curve_results)

print("\n" + "="*60)
print("SUMMARY RESULTS")
print("="*60)
print(f"\nTotal rows in df_results_hybrid: {len(df_results_hybrid)}")

# View only the important columns
df_results_hybrid[['split', 'language', 'method', 'Recall@10', 'MRR@10']]


In [ ]:
sns.set_style("whitegrid")
sns.set_palette("husl")

g = sns.FacetGrid(
    df_alpha_curve,
    col="language",
    col_wrap=3,        
    height=3.5,
    sharey=True,
    sharex=True
)

g.map_dataframe(
    sns.lineplot,
    x="alpha",
    y="val_MRR@10",
    marker="o",
    linewidth=2.5,
    markersize=8,
    color="#2c3e50"
)

g.set_axis_labels(r"$\alpha$ (BM25 weight)", "Validation MRR@10", fontsize=11, fontweight='bold')
g.set_titles("Language: {col_name}", fontsize=12, fontweight='bold')
g.figure.suptitle(
    "Hybrid α Parameter Tuning Across Languages",
    fontsize=15,
    fontweight="bold",
    y=1.00
)

g.figure.subplots_adjust(wspace=0.3, hspace=0.4)

for ax in g.axes.flatten():
    ax.grid(alpha=0.3, linestyle='--')
    ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()


In [ ]:
sns.set_style("whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Three Retrieval Methods: Comprehensive Performance Comparison', fontsize=14, fontweight='bold', y=1.00)

# Left: Overall metrics comparison
ax1 = axes[0]
method_order = ["BM25", "LaBSE", "Hybrid(tuned)"]
colors_methods = ["#3498db", "#e74c3c", "#2ecc71"]

# Get average scores for each method
method_stats = df_results_hybrid.groupby('method')[['Recall@10', 'MRR@10']].mean().reindex(method_order)

x = np.arange(len(method_order))
width = 0.35

bars1 = ax1.bar(x - width/2, method_stats['Recall@10'], width, label='Recall@10', color='#34495e', alpha=0.85)
bars2 = ax1.bar(x + width/2, method_stats['MRR@10'], width, label='MRR@10', color='#e67e22', alpha=0.85)

ax1.set_ylabel('Score', fontsize=12, fontweight='bold')
ax1.set_xlabel('Retrieval Method', fontsize=12, fontweight='bold')
ax1.set_title('Average Performance: All Metrics', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(method_order, fontsize=11)
ax1.legend(fontsize=11, loc='upper left')
ax1.set_ylim(0, 1.0)
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Right: MRR@10 by method (violin/box style for distribution)
ax2 = axes[1]
violin_data = [df_results_hybrid[df_results_hybrid['method']==m]['MRR@10'].values for m in method_order]

bp = ax2.boxplot(violin_data, labels=method_order, patch_artist=True, widths=0.6)

# Color the boxes
for patch, color in zip(bp['boxes'], colors_methods):
    patch.set_facecolor(color)
    patch.set_alpha(0.85)

ax2.set_ylabel('MRR@10', fontsize=12, fontweight='bold')
ax2.set_xlabel('Retrieval Method', fontsize=12, fontweight='bold')
ax2.set_title('MRR@10 Distribution Across Languages', fontsize=12, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
ax2.set_ylim(0, 1.0)

plt.tight_layout()
plt.show()


In [ ]:
sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(13, 6))

df_mono = df_results_hybrid[
    (df_results_hybrid["split"] == "monolingual") &
    (df_results_hybrid["method"].isin(["BM25", "LaBSE", "Hybrid(tuned)"]))
]

sns.barplot(
    data=df_mono,
    x="language",
    y="MRR@10",
    hue="method",
    hue_order=["BM25", "LaBSE", "Hybrid(tuned)"],
    palette=["#3498db", "#e74c3c", "#2ecc71"],
    width=0.7,
    errorbar=None,
    ax=ax
)

ax.set_xlabel("Language", fontsize=12, fontweight='bold')
ax.set_ylabel("MRR@10", fontsize=12, fontweight='bold')
ax.set_title("Monolingual Retrieval Performance by Language", fontsize=13, fontweight='bold')
ax.set_ylim(0, 1.0)
ax.legend(title="Method", fontsize=11, title_fontsize=12, loc='lower right', framealpha=0.95)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(13, 6))

df_cross = df_results_hybrid[
    (df_results_hybrid["split"] == "cross_lingual") &
    (df_results_hybrid["method"].isin(["BM25", "LaBSE", "Hybrid(tuned)"]))
]

sns.barplot(
    data=df_cross,
    x="language",
    y="MRR@10",
    hue="method",
    hue_order=["BM25", "LaBSE", "Hybrid(tuned)"],
    palette=["#3498db", "#e74c3c", "#2ecc71"],
    width=0.7,
    errorbar=None,
    ax=ax
)

ax.set_xlabel("Language", fontsize=12, fontweight='bold')
ax.set_ylabel("MRR@10", fontsize=12, fontweight='bold')
ax.set_title("Cross-Lingual Retrieval Performance by Language", fontsize=13, fontweight='bold')
ax.set_ylim(0, 1.0)
ax.legend(title="Method", fontsize=11, title_fontsize=12, loc='lower right', framealpha=0.95)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## Neural Reranking of Hybrid Top-K Candidates

The following part implements a **two-stage retrieval + neural reranking pipeline**.

**Stage 1 — Candidate generation (Hybrid retrieval):**
For each query, it first builds a candidate set using a tuned hybrid ranker that combines lexical signals (BM25) and semantic signals (dense embeddings) with a mixing weight α, previously tuned. This stage is designed to be efficient and to retrieve a reasonably large shortlist (e.g., top-100) from which the true relevant document is likely to appear.

**Stage 2 — Reranking with a Cross-Encoder:**
For each query, the code reranks the shortlisted documents using a multilingual CrossEncoder [amberoad/bert-multilingual-passage-reranking-msmarco](https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco). The cross-encoder scores each (query, document) pair jointly, allowing deep token-level interactions. To keep CPU inference feasible, it uses batching, limits PyTorch threads, restricts maximum sequence length, and truncates documents.

Finally, it evaluates the reranked top results using Recall@10 and MRR@10, and stores results in a frozen CSV.

**Benefits:** 
- Higher ranking quality (especially MRR@10): the cross-encoder models query–document token interactions directly, improving the ordering of top results.
- Efficiency at scale: expensive neural scoring is applied only to a small candidate set (Top-K), not the whole collection.
- Better multilingual robustness: a multilingual reranker can handle cross-lingual mismatches (vocabulary/phrasing differences) more effectively than pure lexical retrieval.

In [ ]:
from sentence_transformers import CrossEncoder

# Limits the number of threads for PyTorch to avoid CPU overload
try:
    import torch
    torch.set_num_threads(min(4, os.cpu_count() or 1))
except Exception:
    pass

# Multilingual reranker model
reranker = CrossEncoder(
    "amberoad/bert-multilingual-passage-reranking-msmarco",
    device="cpu",
    max_length=256  
)

In [ ]:

def truncate_text(text, max_chars=400):
    """
    Truncate text to max_chars, cutting at last space before limit.
    Benefits: 
    - preserves whole words
    - avoids cutting in the middle of a word
    - avoids very short truncations if no space found
    """
    if not text:
        return ""
    t = text.strip()
    if len(t) <= max_chars:
        return t
    # cut at last space before max_chars
    cut = t[:max_chars]
    last_space = cut.rfind(" ")
    return cut if last_space < 50 else cut[:last_space]


In [ ]:
def rerank_topk_crossencoder(
    reranker,
    query_text,
    doc_ids,
    documents,
    batch_size=8,
    show_progress=False
):
    # Select only valid documents with non-empty text
    valid_doc_ids = []
    valid_texts = []

    for d in doc_ids:
        dt = documents.get(d, "")
        if isinstance(dt, str) and dt.strip() != "":
            valid_doc_ids.append(d)
            valid_texts.append((query_text, dt))

    # If no valid documents, return empty
    if not valid_doc_ids:
        return [], []

    # Predict - the model returns LOGITS [non_relevant, relevant]
    logits = reranker.predict(
        valid_texts,
        batch_size=batch_size,
        show_progress_bar=show_progress
    )

    logits = np.asarray(logits, dtype=np.float32)
    
    # Manage different logits to scores
    if logits.ndim == 2 and logits.shape[1] == 2:
        # calculate probabilities from logits (2 classes) using softmax
        exp_logits = np.exp(logits)
        probs = exp_logits / exp_logits.sum(axis=1, keepdims=True)
        scores = probs[:, 1] # take the "relevant" class probability
    else:
        # Fallback: if single logit per pair, use it directly
        scores = logits.flatten()
    
    scores = scores.flatten()
    
    # Ensure lengths match using minimum length
    if len(scores) != len(valid_doc_ids):
        min_len = min(len(scores), len(valid_doc_ids))
        scores = scores[:min_len]
        valid_doc_ids = valid_doc_ids[:min_len]

    # if we still have no valid scores/docs, return empty
    if len(scores) == 0 or len(valid_doc_ids) == 0:
        return [], []

    # Order by descending scores, first get highest scores
    order = np.argsort(-scores).tolist()

    # Apply the order to valid_doc_ids
    reranked_doc_ids = [valid_doc_ids[i] for i in order]
    reranked_scores  = [float(scores[i]) for i in order]

    return reranked_doc_ids, reranked_scores

In [ ]:
def strip_language_prefix(x):
    """
    'de_1' -> '1'
    'en_english_22626#23' -> 'english_22626#23' (if present)
    If no prefix, returns as-is.
    """
    x = str(x)
    if "_" in x:
        a, b = x.split("_", 1)
        if len(a) in (2, 3):  
            return b
    return x

In [ ]:
def evaluate_dictqrels(retrieved, qrels_eval, K=10):
    """
    retrieved: {qid: [doc1, doc2, ...]}  (doc ids CLEAN)
    qrels_eval: {qid: {docid: rel_int}}
    """
    recalls = []
    rr_list = []

    for qid, top_docs in retrieved.items():
        rels = qrels_eval.get(qid, {})
        if not rels:
            recalls.append(0.0)
            rr_list.append(0.0)
            continue

        rel_set = {d for d, r in rels.items() if r > 0}

        # Recall@K
        hit = any(d in rel_set for d in top_docs[:K])
        recalls.append(1.0 if hit else 0.0)

        # MRR@K
        rr = 0.0
        for rank, d in enumerate(top_docs[:K], start=1):
            if d in rel_set:
                rr = 1.0 / rank
                break
        rr_list.append(rr)

    return float(np.mean(recalls)), float(np.mean(rr_list))


In [ ]:
def normalize_qrels_single_id(qrels):
    """
    Your qrels: { 'de_1': 'de_1', ... }
    Convert to: { 'de_1': {'1': 1} }  (doc ids CLEAN)
    Keep qid keys ORIGINAL (with prefix), because evaluate() expects them.
    """
    qrels_eval = {}
    for qid, rel_doc in qrels.items():
        qid_original = str(qid)
        rel_doc_clean = strip_language_prefix(rel_doc)
        qrels_eval[qid_original] = {rel_doc_clean: 1}
    return qrels_eval

In [ ]:

def sanity_check(
    queries_map,
    queries_map_clean,
    docs_map_clean,
    qrels,
    pack,
    alpha,
    K_RERANK=50,
    CANDIDATE_N=200
):
    print("\n==================== SANITY CHECK ====================")

    missing_queries = []
    missing_rel_docs = []
    hybrid_docs_missing = []

    for qid_original, rel_doc_original in qrels.items():
        qid_clean = strip_language_prefix(qid_original)
        rel_doc_clean = strip_language_prefix(rel_doc_original)

        # ---------------------------------------------------
        # 1. Check: query text exists
        # ---------------------------------------------------
        qtext = queries_map.get(qid_original) or queries_map_clean.get(qid_clean)
        if not qtext:
            missing_queries.append((qid_original, qid_clean))
        
        # ---------------------------------------------------
        # 2. Check: relevant doc exists in docs_map_clean
        # ---------------------------------------------------
        if rel_doc_clean not in docs_map_clean:
            missing_rel_docs.append((qid_original, rel_doc_clean))

        # ---------------------------------------------------
        # 3. Check: docs returned by HYBRID exist in docs_map_clean
        # ---------------------------------------------------
        top_docs = hybrid_topk_for_query(
            qid=qid_original,
            alpha=alpha,
            pack=pack,
            K=K_RERANK,
            candN=CANDIDATE_N
        )
        
        top_docs_clean = [strip_language_prefix(d) for d in top_docs]

        for d in top_docs_clean:
            if d not in docs_map_clean:
                hybrid_docs_missing.append((qid_original, d))

    # ---------------------------------------------------
    # Print results
    # ---------------------------------------------------
    print("\n--- Missing queries (no text found) ---")
    if missing_queries:
        for q in missing_queries[:10]:
            print(" ", q)
        print(f"TOTAL missing queries: {len(missing_queries)}")
    else:
        print("OK ✓")

    print("\n--- Relevant documents missing in docs_map_clean ---")
    if missing_rel_docs:
        for q in missing_rel_docs[:10]:
            print(" ", q)
        print(f"TOTAL missing relevant docs: {len(missing_rel_docs)}")
    else:
        print("OK ✓")

    print("\n--- Docs returned by hybrid but NOT in docs_map_clean ---")
    if hybrid_docs_missing:
        for q in hybrid_docs_missing[:10]:
            print(" ", q)
        print(f"TOTAL hybrid docs missing: {len(hybrid_docs_missing)}")
    else:
        print("OK ✓")

    print("\n==================== END CHECK ====================\n")

    return {
        "missing_queries": missing_queries,
        "missing_rel_docs": missing_rel_docs,
        "hybrid_docs_missing": hybrid_docs_missing
    }


In [ ]:
K_RERANK = 100
K_FINAL  = 10

METHOD_NAME = "Hybrid(tuned)+Rerank(amberoad-mBERT)"
RERANK_MODEL = "amberoad/bert-multilingual-passage-reranking-msmarco"
RERANK_DEVICE = "cpu"
RERANK_MAX_LEN = 256       # set in CrossEncoder
DOC_TRUNC_CHARS = 400      # truncate documents to this length before reranking
RERANK_BATCH = 8

# Path of the shared, versionable file (to be pushed)
FROZEN_DIR = "results/rerank"
FROZEN_PATH = os.path.join(FROZEN_DIR, "df_results_rerank_frozen.csv")
os.makedirs(FROZEN_DIR, exist_ok=True)

FORCE_RERANK = False  # set True only if you want to recompute

if (not FORCE_RERANK) and os.path.exists(FROZEN_PATH) and os.path.getsize(FROZEN_PATH) > 0:
    df_results_rerank = pd.read_csv(FROZEN_PATH)
    print(f"Loaded frozen rerank results: {FROZEN_PATH}")
else:
    print("Frozen rerank results not found (or FORCE_RERANK=True). Running reranking...")

    alpha_map = {
        (r["split"], r["language"]): float(r["alpha"])
        for _, r in df_results_hybrid[df_results_hybrid["method"]=="Hybrid(tuned)"].iterrows()
    }

    rerank_results = []

    for split in CORPUS_SPLITS:
        for lang_code in SELECTED_LANGUAGES:
            key = (split, lang_code)
            if key not in SCORE_CACHE:
                continue

            pack = SCORE_CACHE[key]
            needed = ["qrels", "bm25_topk_docs", "bm25_topk_scores", "dense_topk_docs", "dense_topk_scores"]
            if any(k not in pack for k in needed):
                continue

            alpha = alpha_map.get((split, lang_code), 0.3)
            qrels = pack["qrels"]
            query_ids = list(qrels.keys())
            query_ids = query_ids[:200]

            lang_path = os.path.join(BASE_DATA_DIR, split, lang_code, "train.jsonl")

            
            # 1. Build set of needed document IDs
            needed_ids = set()

            for qid in query_ids:
                qid = str(qid)
                rel_doc = strip_language_prefix(qrels[qid])
                needed_ids.add(rel_doc)

                # top-K hybrid: ID with prefix -> clean
                top_docs = hybrid_topk_for_query(
                    qid=qid,
                    alpha=alpha,
                    pack=pack,
                    K=K_RERANK,
                    candN=CANDIDATE_N
                )

                clean = [strip_language_prefix(d) for d in top_docs]
                needed_ids.update(clean)

            print(f"Needed documents: {len(needed_ids)}")

            # Load the exact documents from JSONL used for BM25/Dense
            data = load_jsonl_robust(lang_path, max_items=MAX_ITEMS)

            docs_map = {}
            queries_map = {}

            # Build maps with prefixed IDs
            for item in data:
                raw_id = str(item["_id"])
                doc_id = f"{lang_code}_{raw_id}"
                # Only keep needed documents
                queries_map[doc_id] = item.get("query", "")
                docs_map[doc_id] = (item.get("title", "") + " " + item.get("text", "")).strip()
            
            # Clean docs_map (without prefixes - clean IDs)
            docs_map_clean = {
                strip_language_prefix(k): truncate_text(v, max_chars=DOC_TRUNC_CHARS)
                for k, v in docs_map.items()
            }
            
            # Sanity check: ensure relevant doc is in docs_map_clean
            allowed = set(docs_map_clean.keys())
            example_qid = query_ids[0]
            rel_clean = strip_language_prefix(qrels[example_qid])
            print("Check overlap:", rel_clean in allowed)
            
            # Clean queries map
            queries_map_clean = {
                strip_language_prefix(k): v
                for k, v in queries_map.items()
            }

            print(f"\n{'='*70}")
            print(f"RERANKING {split}/{lang_code} | alpha={alpha:.2f}")
            print(f"queries loaded: {len(queries_map)} | docs loaded: {len(docs_map)}")
            print(f"{'='*70}")

            # Prepare qrels for evaluation
            qrels_eval = normalize_qrels_single_id(qrels)

            # Sanity check
            check = sanity_check(
                queries_map=queries_map,
                queries_map_clean=queries_map_clean,
                docs_map_clean=docs_map_clean,
                qrels=qrels,
                pack=pack,
                alpha=alpha,
                K_RERANK=K_RERANK,
                CANDIDATE_N=CANDIDATE_N
            )

            reranked_runs = {}
            t0_all = time.time()

            # Rerank each query
            for i, qid_original in enumerate(query_ids):
                qid_original = str(qid_original)
                qid_clean = strip_language_prefix(qid_original)

                qtext = queries_map.get(qid_original) or queries_map_clean.get(qid_clean)
                if not qtext:
                    reranked_runs[qid_original] = []
                    continue

                # Stage 1: HYBRID to get candidates
                top_docs = hybrid_topk_for_query(
                    qid=qid_original,
                    alpha=alpha,
                    pack=pack,
                    K=K_RERANK,
                    candN=CANDIDATE_N
                )

                if not top_docs:
                    reranked_runs[qid_original] = []
                    continue

                # Clean IDs to match docs_map format (WITHOUT prefix)
                top_docs_clean = [strip_language_prefix(d) for d in top_docs]
                valid_docs = [d for d in top_docs_clean if d in docs_map_clean]
                print(f"Query {qid_original}: {len(valid_docs)} documenti validi per rerank")

                if len(valid_docs) > 0:
                    print(f"  Primo doc: {valid_docs[0]}, testo: {len(docs_map_clean.get(valid_docs[0], ''))} chars")

                if not valid_docs:
                    # Fallback su BM25: use only BM25 candidates that are valid
                    bm25_docs = pack["bm25_topk_docs"].get(qid_original, [])
                    bm25_docs_clean = [strip_language_prefix(d) for d in bm25_docs]
                    valid_docs = [d for d in bm25_docs_clean if d in docs_map_clean]

                if not valid_docs:
                    # if still no valid docs, skip
                    reranked_runs[qid_original] = []
                    continue

                # verify if documents are valid
                if len(valid_docs) == 0:
                    reranked_runs[qid_original] = []
                    continue

                # Stage 2: Rerank (CPU-friendly, small batches)
                rr_docs, rr_scores = rerank_topk_crossencoder(
                    reranker=reranker,
                    query_text=qtext,
                    doc_ids=valid_docs,
                    documents=docs_map_clean,
                    batch_size=8,
                    show_progress=False
                )

                # If no reranked docs, skip
                if not rr_docs:
                    reranked_runs[qid_original] = []
                    continue
                # Keep only top-K_FINAL
                reranked_runs[qid_original] = rr_docs[:K_FINAL]

                # Print every 10 query for debugging
                if i % 10 == 0:
                    print(f"Query {qid_original}: {len(valid_docs)} documenti → {len(rr_docs)} dopo rerank")
                    if rr_scores:
                        print(f"  Score range: [{min(rr_scores):.4f}, {max(rr_scores):.4f}]")
                        print(f"  Top-3 scores: {rr_scores[:3]}")
                else:
                    print(f"Query {qid_original}: top score = {rr_scores[0]:.4f}" if rr_scores else f"Query {qid_original}: no scores")

                # Log every 50 queries for execution time
                if (i + 1) % 50 == 0:
                    elapsed = time.time() - t0_all
                    print(f"[{i+1}/{len(query_ids)}] elapsed: {elapsed/60:.1f} min")
                    print("-" * 50)

            # Evaluate
            recall_r, mrr_r = evaluate_dictqrels(
                reranked_runs, qrels_eval, K=K_FINAL
            )

            print(f"\n>>> RESULT {split}/{lang_code}")
            print(f"Recall@10 = {recall_r:.4f}")
            print(f"MRR@10    = {mrr_r:.4f}")

            # Store results
            rerank_results.append({
                "split": split,
                "language": lang_code,
                "Recall@10": float(recall_r),
                "MRR@10": float(mrr_r),
                "method": METHOD_NAME,
                "alpha": float(alpha),
                "candN": int(CANDIDATE_N),
                "rerankK": int(K_RERANK),

                # metadata useful for reproducibility
                "rerank_model": RERANK_MODEL,
                "device": RERANK_DEVICE,
                "max_length": int(RERANK_MAX_LEN),
                "doc_trunc_chars": int(DOC_TRUNC_CHARS),
                "batch_size": int(RERANK_BATCH),
            })


            gc.collect()

            df_results_rerank = pd.DataFrame(rerank_results)
            df_results_rerank

In [ ]:
# freeze the rerank results

FROZEN_DIR = "results/rerank"
FROZEN_PATH = os.path.join(FROZEN_DIR, "df_results_rerank_frozen.csv")
os.makedirs(FROZEN_DIR, exist_ok=True)

if "df_results_rerank" not in globals() or df_results_rerank is None or df_results_rerank.empty:
    raise RuntimeError("df_results_rerank not available or empty. Run the rerank cell first.")

df_results_rerank.to_csv(FROZEN_PATH, index=False)
print(f" Saved frozen rerank results to: {FROZEN_PATH}")


In [ ]:
CSV_PATH = "results/rerank/df_results_rerank_frozen.csv"  

df = pd.read_csv(CSV_PATH)
cols_preferred = [
    "split", "language", "Recall@10", "MRR@10", "alpha", "candN", "rerankK", "device", "max_length", "doc_trunc_chars", "batch_size", "method"
]
cols = [c for c in cols_preferred if c in df.columns]
df_view = df[cols].copy() if cols else df.copy()

sort_cols = [c for c in ["split", "language"] if c in df_view.columns]
if "MRR@10" in df_view.columns:
    df_view = df_view.sort_values(by=sort_cols + ["MRR@10"], ascending=[True]*len(sort_cols) + [False])
else:
    df_view = df_view.sort_values(by=sort_cols) if sort_cols else df_view

display(df_view)




# Results and Conclusions

## Summary of Results

Our experiments comparing BM25 (sparse retrieval), LaBSE (dense retrieval), and Hybrid methods across 5 languages (German, French, Spanish, English, Chinese) in both monolingual and cross-lingual settings yielded clear findings:

### Overall Performance
- **LaBSE Recall@10**: 84.9% (vs. BM25: 66.9%)
- **LaBSE MRR@10**: 0.708 (vs. BM25: 0.575)
- **Hybrid Recall@10**: 87.9% (best overall)
- **Hybrid MRR@10**: 0.761 (best overall)
- **Head-to-head**: LaBSE wins 7 out of 8 language comparisons against BM25 (87.5%)

### Language-Specific Performance

**Monolingual Results:**
- **German**: LaBSE (88.8% Recall, 0.799 MRR) significantly outperforms BM25
- **English**: LaBSE (93.0% Recall, 0.846 MRR) - highest performance across all languages
- **Spanish**: LaBSE (86.1% Recall, 0.734 MRR) shows strong semantic understanding
- **French**: LaBSE (76.1% Recall, 0.616 MRR) - competitive with BM25

**Cross-Lingual Results:**
- **Chinese**: LaBSE (85.4% Recall, 0.679 MRR) - crucial for non-alphabetic systems
- **German**: LaBSE (85.2% Recall, 0.684 MRR) - consistent cross-lingual performance
- **French**: LaBSE (79.4% Recall, 0.626 MRR) - handles language mismatch well
- **Spanish**: LaBSE (85.3% Recall, 0.678 MRR) - strong multilingual alignment

### Hybrid Method Performance

The tuned hybrid approach (combining BM25 and LaBSE with optimized α parameter) achieved:
- **German (monolingual)**: 92.5% Recall, 0.875 MRR with α=0.30
- **Spanish (monolingual)**: 90.5% Recall, 0.826 MRR with α=0.30
- **English (monolingual)**: 95.0% Recall, 0.898 MRR with α=0.20

The α parameter varied by language (0.20-0.50), indicating language-specific optimal balances between lexical and semantic matching.

### Key Observations

1. **Cross-lingual superiority**: LaBSE dramatically outperforms BM25 on cross-lingual tasks
   - BM25 struggles when query and document are in different languages (limited by vocabulary mismatch)
   - LaBSE's multilingual embeddings enable effective cross-lingual matching through semantic alignment
   - All cross-lingual tasks showed >79% Recall@10 with LaBSE

2. **Monolingual performance**: LaBSE excels even in monolingual scenarios
   - Contrary to expectations, LaBSE wins on English monolingual (93% vs BM25's lower performance)
   - Semantic matching captures paraphrases and synonyms that lexical matching misses
   - Only competitive on French monolingual, where both methods perform similarly

3. **Hybrid advantage**: Combining methods yields best results
   - Hybrid consistently outperforms both individual methods
   - Language-specific tuning of α shows importance of balancing lexical and semantic signals
   - Lower α values (favoring dense retrieval) work better for Germanic languages
   - Higher α values (more BM25 weight) needed for Romance languages

4. **Language-specific insights**:
   - **Largest improvements**: English (monolingual) shows highest absolute performance
   - **Cross-lingual strength**: Chinese benefits most from semantic matching (no alphabet overlap with English queries)
   - **Consistent performance**: German shows stable high performance across both splits
   - **Optimization opportunities**: French and Spanish show room for improvement with better α tuning

## Conclusions

Dense retrieval using multilingual embeddings (LaBSE) significantly outperforms traditional sparse methods (BM25) for multilingual information retrieval in both monolingual and cross-lingual scenarios. The ability to capture semantic similarity across languages makes dense retrieval the strongly preferred approach for multilingual IR systems.

The hybrid approach combining BM25 and LaBSE scores with tuned fusion weights achieves the best overall performance (87.9% Recall@10, 0.761 MRR@10), demonstrating that lexical and semantic signals complement each other effectively. Language-specific α tuning is crucial, as optimal balance varies significantly across language families.

### Practical Implications

1. **For production systems**: Deploy hybrid methods with language-specific tuning for optimal results
2. **For cross-lingual search**: LaBSE alone provides strong baseline (85%+ recall)
3. **For monolingual English**: Hybrid with low α (0.2-0.3) maximizes performance
4. **For resource-constrained settings**: LaBSE-only provides excellent tradeoff between performance and complexity

## Limitations

- **Dataset size**: Evaluated on 1,000 documents per language (limited by memory constraints on M3 Mac with 16GB RAM)
- **Language coverage**: Only 5 languages tested from the available 40+ in SWIM-IR dataset
- **Hardware constraints**: Memory limitations prevented larger batch sizes and full dataset evaluation; used batch_size=1 for encoding
- **Single dense model**: Only evaluated LaBSE; other multilingual models (mE5, mMiniLM) might show different performance characteristics
- **Evaluation setup**: Each query has exactly one relevant document; real-world scenarios may have multiple relevant documents
- **Validation set size**: Limited validation data (20% split) may affect reliability of α tuning

## Future Work

1. **Extended language coverage**: Evaluate on all 40+ languages in SWIM-IR, including low-resource languages
2. **Alternative fusion methods**: Explore Reciprocal Rank Fusion (RRF) and learned-to-rank approaches
3. **Query expansion**: Investigate multilingual query expansion techniques
4. **Fine-tuning**: Adapt LaBSE on domain-specific multilingual data for specialized applications
5. **Efficiency optimization**: Implement FAISS or approximate nearest neighbor methods for larger-scale deployment